# OSB Vector Database Example

This notebook demonstrates how to create and use a vector database from Oversight Board full text data using Buttermilk's ChromaDB integration.

## Overview

We'll show how to:
1. Load OSB JSON data using existing data loaders
2. Generate embeddings and create a ChromaDB vector store
3. Use the generic RAG agent for interactive question answering
4. Demonstrate semantic search capabilities

This example uses the generic infrastructure that works with any JSON dataset.

## 1. Configuration Setup

First, let's set up the configuration for our OSB vector database pipeline.

In [1]:
from rich import print
from rich.pretty import pprint
import asyncio
import json
from pathlib import Path
import hydra
from hydra import compose, initialize_config_dir
from omegaconf import DictConfig, OmegaConf

# Buttermilk imports
from buttermilk import logger
from buttermilk.data.vector import ChromaDBEmbeddings, InputDocument, DefaultTextSplitter
from buttermilk.agents.rag.rag_agent import RagAgent
from buttermilk._core.config import AgentConfig, DataSourceConfig

from buttermilk.utils.nb import init
from buttermilk._core.dmrc import get_bm, set_bm
from buttermilk._core.config import DataSourceConfig
from buttermilk._core.types import Record
from buttermilk.data.loaders import create_data_loader
from buttermilk.data.vector import ChromaDBEmbeddings, InputDocument, DefaultTextSplitter, list_to_async_iterator

# Initialize Buttermilk
cfg = init(job="osb_vectorise", overrides=["+storage=osb"])
bm = get_bm()

print("🚀 Buttermilk initialized for JSON-to-Vector tutorial")
pprint(cfg.storage)


2025-06-16 19:29:56 [] INFO bm_init.py:745 Logging set up for run: platform='local' name='bm_api' job='osb_vectorise' run_id='20250616T0929Z-F6n2-docker-desktop-debian' ip=None node_name='docker-desktop' save_dir='/tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian' flow_api=None. Save directory: /tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian


Initialized Buttermilk (bm) with configuration:

{
    'platform': 'local',
    'name': 'bm_api',
    'job': 'osb_vectorise',
    'run_id': '20250616T0929Z-F6n2-docker-desktop-debian',
    'node_name': 'docker-desktop',
    'save_dir': '/tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian',
    'connections': [],
    'secret_provider': {
        'type': 'gcp',
        'project': 'prosocial-443205',
        'models_secret': 'dev__llm__connections',
        'credentials_secret': 'dev__shared_credentials'
    },
    'logger_cfg': {'type': 'gcp', 'project': 'prosocial-443205', 'location': 'us-central1', 'verbose': False},
    'pubsub': {
        'type': 'gcp',
        'project': 'prosocial-443205',
        'jobs_subscription': 'jobs-sub',
        'status_subscription': 'flow-sub',
        'status_topic': 'flow',
        'jobs_topic': 'jobs'
    },
    'clouds': [
        {'type': 'gcp', 'project': 'prosocial-443205', 'quota_project_id': 'prosocial-443205'},
        {
            'type': 'vertex',
            'project': 'prosocial-443205',
            'region': 'us-central1',
            'location': 'us-central1',
            'bucket': 'prosocial-de'
        }
    ],
    'tracing': {'enabled': True, 'api_key': '', 'provider': 'weave', 'otlp_headers': {}},
    'datasets': {},
    'save_dir_base': '/tmp/tmpdvnhck_1'
}

2025-06-16 19:29:56 [] INFO nb.py:59 Starting interactive run for bm_api job osb_vectorise in notebook


🚀 Buttermilk initialized for JSON-to-Vector tutorial

{'osb_json': {'type': 'gcs', 'path': 'gs://prosocial-public/osb/03_osb_fulltext_summaries.json', 'columns': {'record_id': 'id', 'content': 'full_text', 'metadata': {'title': 'title', 'case_number': 'case_number', 'url': 'url', 'summary': 'summary'}}}, 'osb_vector': {'type': 'chromadb', 'persist_directory': 'gs://prosocial-public/osb/chromadb', 'collection_name': 'osb_fulltext', 'embedding_model': 'gemini-embedding-001', 'dimensionality': 3072}}

2025-06-16 19:29:56 [] INFO save.py:641 Successfully dumped data to local disk (JSON): /tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian/tmpxbjvgc6j.json.
2025-06-16 19:29:56 [] INFO save.py:215 Successfully saved data using dump_to_disk to: /tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian/tmpxbjvgc6j.json.
2025-06-16 19:29:56 [] INFO bm_init.py:831 {'message': 'Successfully saved data to: /tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian/tmpxbjvgc6j.json', 'uri': '/tmp/tmpdvnhck_1/bm_api/osb_vectorise/20250616T0929Z-F6n2-docker-desktop-debian/tmpxbjvgc6j.json', 'run_id': '20250616T0929Z-F6n2-docker-desktop-debian'}


## 2. Initialize Components

Let's create the storage, vector store, and text splitter components.

In [ ]:
# Now we can use the clean BM API for all storage types
source = bm.get_storage(cfg.storage.osb_json)
vectorstore = bm.get_storage(cfg.storage.osb_vector)

# Create text splitter
chunker = DefaultTextSplitter(chunk_size=2000, chunk_overlap=500)

print("✅ All storage components initialized via BM API")
print(f"Source: {type(source).__name__}")
print(f"Vector store: {type(vectorstore).__name__}")
print(f"Text splitter: {type(chunker).__name__}")

In [ ]:
# Ensure cache is initialized for remote ChromaDB
await vectorstore.ensure_cache_initialized()

print("✅ ChromaDB cache ready for processing")

## 3. Load OSB Data

Now let's load the OSB JSON data and convert it to InputDocument format.

In [ ]:
# Load actual OSB data using the data loader
print("📥 Loading OSB data from GCS...")

# Create data loader from source storage
from buttermilk.data.loaders import create_data_loader
from buttermilk._core.config import DataSourceConfig

# Convert storage config to data source config
source_config = DataSourceConfig(**cfg.storage.osb_json)
data_loader = create_data_loader(source_config)

# Get records from the data loader
records = []
async for record in data_loader.load():
    records.append(record)
    if len(records) >= 3:  # Just process first 3 for demo
        break

print(f"✅ Loaded {len(records)} OSB records")

# Convert Records to InputDocument format
documents = []
for record in records:
    doc = InputDocument(
        record_id=record.record_id,
        title=record.metadata.get('title', 'Untitled'),
        full_text=record.content if isinstance(record.content, str) else str(record.content),
        metadata=record.metadata,
        file_path="",
    )
    documents.append(doc)
    print(f"- {doc.record_id}: {doc.title[:50]}...")

## 4. Process Documents: Chunking and Embedding

Now we'll chunk the documents and generate embeddings.

In [ ]:
async def process_documents():
    """Process documents through the complete pipeline."""
    processed_docs = []

    for doc in documents:
        print(f"\nProcessing document: {doc.record_id}")

        # Step 1: Chunk the document
        chunked_doc = await chunker.process(doc)
        if not chunked_doc:
            print(f"Failed to chunk document {doc.record_id}")
            continue

        print(f"Created {len(chunked_doc.chunks)} chunks")

        # Step 2: Generate embeddings and store
        processed_doc = await vectorstore.process(chunked_doc)
        if processed_doc:
            processed_docs.append(processed_doc)
            print(f"Successfully embedded and stored document {doc.record_id}")
        else:
            print(f"Failed to process document {doc.record_id}")

    return processed_docs


# Run the processing pipeline
processed_documents = await process_documents()
print(f"\nProcessed {len(processed_documents)} documents successfully")


## 5. Initialize RAG Agent

Now let's create a generic RAG agent to query our OSB vector database.

In [ ]:
# Create data source configuration for the RAG agent
data_config = DataSourceConfig(
    type="chromadb", persist_directory="./data/osb_chromadb", collection_name="osb_fulltext", embedding_model="text-embedding-005", dimensionality=768
)

# Create agent configuration
agent_config = AgentConfig(
    role="RESEARCHER",
    agent_obj="RagAgent",
    description="OSB Research Assistant",
    data={"osb_vector": data_config},
    variants={"model": "gemini-1.5-flash"},
    parameters={"template": "rag_research", "n_results": 10, "no_duplicates": False, "max_queries": 3},
)

# Initialize the RAG agent
rag_agent = RagAgent(**agent_config.model_dump())
print("RAG agent initialized successfully")


## 6. Semantic Search Examples

Let's demonstrate the semantic search capabilities of our OSB vector database.

In [ ]:
async def search_osb_database(queries):
    """Search the OSB database with multiple queries."""
    print("\n=== OSB Database Search Results ===")

    results = await rag_agent.fetch(queries)

    for i, (query, result) in enumerate(zip(queries, results)):
        print(f"\n--- Query {i+1}: {query} ---")
        print(f"Found {len(result.results)} relevant chunks")

        if result.results:
            # Show the top result
            top_result = result.results[0]
            print(f"\nTop Result:")
            print(f"Document: {top_result.document_title}")
            print(f"Case Number: {top_result.metadata.get('case_number', 'N/A')}")
            print(f"Text: {top_result.full_text[:300]}...")
        else:
            print("No results found")


# Example search queries
search_queries = [
    "What are the challenges with automated content moderation?",
    "How effective are age verification systems?",
    "What techniques are used to spread misinformation?",
]

await search_osb_database(search_queries)


## 7. Interactive Chat Interface

Now let's create an interactive interface to chat with our OSB knowledge base.

In [ ]:
async def chat_with_osb(user_question):
    """Interactive chat with OSB knowledge base."""
    print(f"\n🔍 User Question: {user_question}")

    # Search for relevant context
    search_results = await rag_agent.fetch([user_question])

    if search_results and search_results[0].results:
        context = search_results[0]
        print(f"\n📚 Found {len(context.results)} relevant documents")

        # Display relevant chunks
        print("\n📋 Relevant Information:")
        for i, result in enumerate(context.results[:3]):  # Show top 3
            print(f"\n{i+1}. {result.document_title} ({result.metadata.get('case_number', 'N/A')})")
            print(f"   {result.full_text[:200]}...")

        # In a real implementation, this would be sent to an LLM for synthesis
        print("\n🤖 AI Response: [In a real implementation, the retrieved context would be sent to an LLM to generate a synthesized response]")
    else:
        print("\n❌ No relevant information found in the OSB database")


# Example chat interactions
example_questions = [
    "What are the main issues with current content moderation approaches?",
    "What recommendations exist for age verification?",
    "How do platforms detect and counter misinformation?",
]

for question in example_questions:
    await chat_with_osb(question)
    print("\n" + "=" * 80)


## 8. Vector Store Analysis

Let's analyze our vector store to understand what we've created.

In [ ]:
# Get collection statistics
collection = vectorstore.collection
count = collection.count()

print(f"\n=== OSB Vector Store Statistics ===")
print(f"Collection Name: {vectorstore.collection_name}")
print(f"Total Chunks: {count}")
print(f"Embedding Dimensions: {vectorstore.dimensionality}")
print(f"Embedding Model: {vectorstore.embedding_model}")

# Get a sample of metadata to understand the structure
sample_results = collection.get(limit=3, include=["metadatas", "documents"])

print(f"\n=== Sample Metadata Structure ===")
if sample_results["metadatas"]:
    sample_metadata = sample_results["metadatas"][0]
    print("Available metadata fields:")
    for key, value in sample_metadata.items():
        print(f"  - {key}: {type(value).__name__} = {str(value)[:50]}...")

print(f"\n=== Storage Locations ===")
print(f"ChromaDB Directory: {vectorstore.persist_directory}")
print(f"Embeddings Directory: {vectorstore.arrow_save_dir}")


## 9. Advanced Search Examples

Let's explore some advanced search patterns and filtering capabilities.

In [ ]:
# Direct ChromaDB queries with metadata filtering
async def advanced_search_examples():
    """Demonstrate advanced search capabilities."""
    print("\n=== Advanced Search Examples ===")

    # 1. Search with metadata filtering
    print("\n1. Search within specific case:")
    results = collection.query(
        query_texts=["content moderation challenges"], n_results=5, where={"case_number": "OSB-2024-001"}, include=["documents", "metadatas"]
    )
    print(f"   Found {len(results['ids'][0]) if results['ids'] else 0} results in OSB-2024-001")

    # 2. Similarity search across all documents
    print("\n2. General similarity search:")
    results = collection.query(query_texts=["artificial intelligence and safety"], n_results=5, include=["documents", "metadatas", "distances"])

    if results["ids"] and results["ids"][0]:
        print(f"   Found {len(results['ids'][0])} results")
        for i, (doc, metadata, distance) in enumerate(zip(results["documents"][0][:3], results["metadatas"][0][:3], results["distances"][0][:3])):
            print(f"   Result {i+1} (similarity: {1-distance:.3f}): {metadata.get('title', 'N/A')}")
            print(f"     {doc[:100]}...")

    # 3. Multi-query search
    print("\n3. Multi-query search:")
    multi_queries = ["platform safety measures", "user protection mechanisms", "digital safety standards"]

    for query in multi_queries:
        results = collection.query(query_texts=[query], n_results=2, include=["metadatas"])
        count = len(results["ids"][0]) if results["ids"] else 0
        print(f"   '{query}': {count} results")


await advanced_search_examples()


## 10. Production Considerations

Here are key considerations for using this in production:

In [ ]:
print(
    """
=== Production Deployment Checklist ===

🔧 Configuration:
   ✓ Use GCS for persist_directory: gs://your-bucket/chromadb
   ✓ Configure appropriate chunk_size for your content
   ✓ Set concurrency based on your compute resources
   ✓ Use production embedding models (text-embedding-004/005)

📊 Performance:
   ✓ Monitor embedding generation costs
   ✓ Implement caching for frequently accessed data
   ✓ Use batch processing for large datasets
   ✓ Configure appropriate timeout values

🔒 Security:
   ✓ Secure GCS bucket access with proper IAM
   ✓ Implement data access controls
   ✓ Audit vector store queries
   ✓ Protect sensitive metadata

🚀 Scalability:
   ✓ Plan for vector store size growth
   ✓ Implement horizontal scaling for embeddings
   ✓ Monitor query performance
   ✓ Set up proper logging and monitoring

🔄 Maintenance:
   ✓ Plan for data updates and reindexing
   ✓ Implement backup strategies
   ✓ Version control for embeddings and metadata
   ✓ Regular quality assessments
"""
)

# Show next steps
print(
    """
=== Next Steps ===

1. Scale to Full Dataset:
   - Use the osb_vectorize.yaml configuration
   - Run: uv run python -m buttermilk.data.vector +run=osb_vectorize

2. Deploy RAG Flow:
   - Use the osb_rag.yaml flow configuration
   - Run: uv run python -m buttermilk.runner.cli +flow=osb_rag +run=api

3. Integrate with Frontend:
   - Use the Buttermilk web interface
   - Connect to WebSocket endpoints for real-time chat

4. Monitor and Optimize:
   - Track query performance
   - Monitor embedding costs
   - Tune chunk sizes and retrieval parameters
"""
)


## Summary

This notebook demonstrated how to:

1. **Load OSB data** using Buttermilk's flexible JSON data loaders
2. **Create embeddings** with the ChromaDBEmbeddings pipeline
3. **Build a vector database** suitable for semantic search
4. **Use the generic RAG agent** for question answering
5. **Perform semantic search** with various query patterns
6. **Analyze the vector store** structure and contents

The key advantage of this approach is that it uses **generic, reusable components** that work with any JSON dataset, not just OSB data. The same patterns can be applied to any document collection for RAG applications.

The infrastructure is **production-ready** with features like:
- Async processing for scalability
- GCS integration for cloud storage
- Configurable chunking and embedding parameters
- Error handling and retry mechanisms
- Comprehensive logging and monitoring

This provides a solid foundation for building sophisticated RAG applications with Buttermilk.